In [1]:
from yggdrasil.examples import yamls as ex_yamls
import yaml
import jupyterlab_nodeeditor as jlne
import pytest

In [3]:
# Improved version of making a JLNE-compliant dictionary from a Yggdrasil Model YAML
# Still semi-hard coded for the Photosynthesis model
# TO-DO : Make more robust, Test on custom models 1-3
def dict_conversion(model_dict):
    # Setup initial dictionary to be filled
    new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], [], model_dict["name"]
    # If the type of the safe load is a dictionary, we know it is a single model
    # If it is a list, then we have multiple models
    new_dict["title"]
    
    # Fill in the Inputs
    for i, inp in enumerate(model_dict["inputs"]):
        new_dict["inputs"].append({'title': inp["name"], 'key': "temp_in" + str(i), 'socket_type': inp["default_file"]["filetype"]})
        
    # Fill in the Outputs, same as inputs with name changes
    for o, out in enumerate(model_dict["outputs"]):
        new_dict["outputs"].append({'title': out["name"], 'key': "temp_out" + str(o), 'socket_type': out["default_file"]["filetype"]})
           
    return new_dict

In [22]:
# By default, load the photosynthesis model
# ps - Node Editor instance that it is added to, default blank
def load_example(ps = None):
    with open(ex_yamls['fakeplant']['python'], "r") as test_model:
        photosynthesis_model = yaml.safe_load(test_model)['model']
        
    print(type(photosynthesis_model))
    ps = ps or jlne.NodeEditor()
    
    # Add in the converted dictionary of the photosynthesis model
    ps.add_component(dict_conversion(photosynthesis_model))
    return ps

In [23]:
load_example()

<class 'dict'>


In [5]:
# Testing Functions

# The first test is just to visually ensure that the model appears in the node editor instance
# This is NOT an explicit function
# Look for Inputs, Outputs, and a proper title

# Test 2 : Verify Editor instance w output
# This is done with JLNE's built in class methods of calling a unique editor instance with the models
def test_instance(test_editor):
    expected_editor = jlne.NodeEditor()
    assert test_editor == expected_editor

# Test 3 : Blank




In [30]:
def load_model(ps = None):
    with open("sample_models/sample_2.yml", "r") as sample:
        sample_model = yaml.safe_load(sample)["model"]
    print(type(sample_model))
    ps = ps or jlne.NodeEditor()
    ps.add_component(dict_conversion(sample_model))
    return ps

In [31]:
load_model()

<class 'dict'>


KeyError: 'inputs'